# BƯỚC 5: XÂY DỰNG PIPELINE HUẤN LUYỆN CHUẨN KHOA HỌC
--- 
Notebook này tập trung vào khâu tiền xử lý dữ liệu và thiết lập hạ tầng huấn luyện, bao gồm:
1. **Dọn dẹp dữ liệu:** Loại bỏ ảnh trùng lặp bằng thuật toán MD5.
2. **Phân chia dữ liệu:** Tách tập Train/Val/Test theo tỉ lệ 80/10/10.

In [6]:
import os, hashlib, shutil, torch, glob
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
import splitfolders # pip install split-folders

# --- CẤU HÌNH ĐƯỜNG DẪN ---
BASE_PATH = r'd:\HUTECH\AI\DeepLearning\DAHS\MangoLeaf'
SRC_PATH = os.path.join(BASE_PATH, 'MangoLeaf_Dataset') # Thư mục gốc chứa 8-9 loại bệnh
DST_PATH = os.path.join(BASE_PATH, 'dataset_scientific_split')

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using Device: {DEVICE}')

Using Device: cuda


### 5.1. MD5 Deduplication (Loại bỏ ảnh trùng lặp)
Đảm bảo tính khách quan bằng cách loại bỏ các bộ dữ liệu bị 'leak' hoặc trùng lặp nội dung.

In [8]:
def remove_duplicates(root_dir):
    unique_hashes = set()
    duplicates = 0
    
    for folder in os.listdir(root_dir):
        folder_path = os.path.join(root_dir, folder)
        if not os.path.isdir(folder_path): continue
        
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            with open(file_path, 'rb') as f:
                file_hash = hashlib.md5(f.read()).hexdigest()
            
            if file_hash in unique_hashes:
                os.remove(file_path)
                duplicates += 1
            else:
                unique_hashes.add(file_hash)
                
    print(f'>>> Đã loại bỏ {duplicates} ảnh trùng lặp hoàn toàn.')

# remove_duplicates(SRC_PATH)

### 5.2. Scientific Data Split (80/10/10)
Chia dữ liệu thành 3 tập riêng biệt: Train (Học), Val (Kiểm định quá trình), và Test (Đánh giá cuối cùng).

In [9]:
if not os.path.exists(DST_PATH):
    splitfolders.ratio(SRC_PATH, output=DST_PATH, seed=42, ratio=(.8, .1, .1))
    print(f'>>> Đã phân chia dữ liệu vào: {DST_PATH}')
else:
    print('>>> Dữ liệu đã được split trước đó.')

>>> Dữ liệu đã được split trước đó.
